# Data Modelling

In [116]:
from pyspark.sql.session import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.clustering import KMeans
from helpers.helper_functions import translate_to_file_string
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.ml import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics

inputFile = translate_to_file_string("./data/Data_Preparation_Result.csv")

def prettyPrint(dm, collArray) :
    rows = dm.toArray().tolist()
    dfDM = spark.createDataFrame(rows,collArray)
    newDf = dfDM.toPandas()
    from IPython.display import display, HTML
    return HTML(newDf.to_html(index=False))

## Create Spark Session

In [117]:
#create a SparkSession
spark = (SparkSession
       .builder
       .appName("DataModelling")
       .getOrCreate())
# create a DataFrame using an ifered Schema 
df = spark.read.option("header", "true") \
       .option("inferSchema", "true") \
       .option("delimiter", ";") \
       .csv(inputFile)   
print(df.printSchema())

root
 |-- Bundesland: string (nullable = true)
 |-- BundeslandIndex: integer (nullable = true)
 |-- Landkreis: string (nullable = true)
 |-- LandkreisIndex: integer (nullable = true)
 |-- Altersgruppe: string (nullable = true)
 |-- AltersgruppeIndex: double (nullable = true)
 |-- Geschlecht: string (nullable = true)
 |-- GeschlechtIndex: double (nullable = true)
 |-- FallStatus: string (nullable = true)
 |-- FallStatusIndex: double (nullable = true)
 |-- Falldatum: string (nullable = true)

None


## Vorbereitung der Daten

### Filtern der Datensätze
Für das Training dieses Modells ist es sinnvoll nur die Fälle zu betrachten, bei den der Ausgang der Corona-Erkrankung bereits bekannt ist ("GENESEN" oder "GESTORBEN"). Daher werden die Fälle mit noch erkrankten Personen herausgefiltert. Ebenfalls muss der FallStatusIndex neu vergeben werden, damit dieses Feature nur noch die Werte 0 oder 1 enthält.

In [118]:
dfNeu = df.filter(df.FallStatus != "NICHTEINGETRETEN").drop("FallStatusIndex")

### FallStatusIndex

In [119]:
indexer = StringIndexer(inputCol="FallStatus", outputCol="FallStatusIndex")
dfReindexed = indexer.fit(dfNeu).transform(dfNeu)

### Ziehen eines Samples
Da der Datensatz sehr groß ist,kann es evt. notwendig sein, nur mit einem kleineren Umfang zu trainieren. Mit Fraction kann an dieser Stelle der Umfang der Stichprobe angepasst werden.

In [120]:
dfsample = dfReindexed.sample(withReplacement=False, fraction=1.0, seed=12334556)

### Undersampling
Ähnlich dem Fraud-Detection-Beispiel von Tara Boyle (2019) ist die Klasse der an Corona-Verstorbenen im vorliegenden Datensatz unterrepresentiert, weshalb man an dieser Stelle von einer Data Imbalance spricht. Dies sieht man wenn man die Anzahl der Todesfälle mit den Anzahl der Genesenen vergleicht.

In [121]:
# Vergleich der Fallzahlen
dfsample.groupBy("FallStatus").count().show()

+----------+-------+
|FallStatus|  count|
+----------+-------+
|   GENESEN|3471830|
| GESTORBEN|  88350|
+----------+-------+



Die meisten Machine Learning Algorithmen arbeiten am Besten wenn die Nummer der Samples in allen Klassen ungefähr die selbe größe haben. Dies konnte auch im Zuge dieser Arbeit bei den unterschiedlichen Regressions-Modellen festgestellt werden. Da die einzelnen Modelle versuchen den Fehler zu reduzieren, haben alle Modelle am Ende für einen Datensatz nur die Klasse Genesen geliefert, da hier die Wahrscheinlichkeit am größten war korrekt zu liegen. 
Um diese Problem zu lösen gibt es zwei Möglichkeiten: Under- und Oversampling. Beides fällt unter den Begriff Resampling
Beim Undersampling werden aus der Klasse mit den meisten Instanzen, Datensätze gelöscht, wohingegen beim Oversampling, der Klasse mit den wenigsten Isntanzen, neue Werte hinzugefügt werden. (Will Badr 2019; Tara Boyle 2019)
Da in diesem Fall ausreichend Datensätze vorhanden sind, bietet sich Ersteres an.

In [122]:
# Ermittlung der Anzahl dr Verstorbenen
dfGestorben = dfsample.filter(dfsample.FallStatus == "GESTORBEN")
anzahlGestorben = dfGestorben.count()
print("Anzahl Gestorben : %s" % anzahlGestorben)

Anzahl Gestorben : 88350


In [123]:
# Ermittlung des Verhätlnisses von Verstorben und Gensen
dfGenesen = dfsample.filter(dfsample.FallStatus == "GENESEN")
anzahlGenesen = dfGenesen.count()
print("Anzahl Genesen : %s" % anzahlGenesen)

ratio = anzahlGestorben / anzahlGenesen
print("Verhältnis : %s" % ratio)



Anzahl Genesen : 3471830
Verhältnis : 0.02544767456931935


In [124]:
# Ziehen eines Samples mit der näherungsweise selben Anzahl wie Verstorbene
dfGenesenSample = dfGenesen.sample(fraction=ratio, seed=12345)

In [125]:
dfGesamtSample = dfGestorben.union(dfGenesenSample)
# Kontrolle
dfGesamtSample.groupBy("FallStatus").count().show()

+----------+-----+
|FallStatus|count|
+----------+-----+
|   GENESEN|88520|
| GESTORBEN|88350|
+----------+-----+



### Splitten in Trainings und Testdaten

In [126]:
splits = dfGesamtSample.randomSplit([0.8, 0.2], 345678)
trainingData = splits[0]
testData = splits[1]

### Aufbau des Feature-Vectors

In [127]:
assembler =  VectorAssembler(outputCol="features", inputCols=["GeschlechtIndex","FallStatusIndex","AltersgruppeIndex", "LandkreisIndex","BundeslandIndex"])

### Aufbau eiens VectorIndexer
Ein VectorIndexer dient zur kategorisierung von Features in einem Vector-Datensatz.

In [128]:
featureIndexer = VectorIndexer(inputCol="features",outputCol="indexedFeatures", maxCategories=10) 
# TODO Max Kategories testen

## Modellierung
### K-Means
K-Means ist eines der meist verwendent Clustering-Algortimen. (Apache Spark 2021)

In [129]:
kmeans = KMeans(featuresCol="indexedFeatures", predictionCol="prediction", seed=122334455) #predictionCol="prediction",

### Pipeline

In [130]:
pipeline = Pipeline(stages=[assembler,featureIndexer, kmeans])

### Evaluator
Das Clustering wird mit einem spezielen ClusteringEvaluator evaluiert.

In [131]:
# Definition des Evaluators
evaluator = ClusteringEvaluator()

### Parametertuning
Eine wichtige Aufgabe beim Machine Learning ist die Auswahl des geeigneten Modells bzw. die passenden Paramter für ein Modell herauszufinden. Letzteres wird auch Parametertuning genannt. Die in Pyspark enthaltene MLLib bietet speziell hierfür ein entsprechende Tooling. Und zwar kann ein CrossValidator bzw. ein TrainValidationSplit verwendet werden. Voraussetzung sind ein Estimator (ein Modell oder eine Pipeline), ein Paramter-Grid und eine Evaluator. Dies ist auch im Zusammenhang mit dem Thema Cross-Validation zu sehen. (Apache Spark 2020a)

In [132]:
paramGrid = ParamGridBuilder()\
    .addGrid(kmeans.k, [3,4,6]) \
    .addGrid(kmeans.maxIter, [10])\
    .build()

### Cross-Validation

In [133]:
# Definition des Cross-Validators 
# num-Folds gibt an in wie viele Datensatz-Paare die Datensätze aufgeteilt werden.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=2,
                          parallelism=2)

#### Training

In [134]:
# Anpassung des Modells und Auswahl der besten Parameter
cvModel = crossval.fit(trainingData)

### Ermitteln der Paramter

In [135]:
kmModel = cvModel.bestModel.stages[2]
print(kmModel.explainParams())
centers = kmModel.clusterCenters()
print("Cluster Centers: ")
for center in centers:
   print(center)

distanceMeasure: the distance measure. Supported options: 'euclidean' and 'cosine'. (default: euclidean)
featuresCol: features column name. (default: features, current: indexedFeatures)
initMode: The initialization algorithm. This can be either "random" to choose random points as initial cluster centers, or "k-means||" to use a parallel variant of k-means++ (default: k-means||)
initSteps: The number of steps for k-means|| initialization mode. Must be > 0. (default: 2)
k: The number of clusters to create. Must be > 1. (default: 2, current: 4)
maxIter: max number of iterations (>= 0). (default: 20, current: 10)
predictionCol: prediction column name. (default: prediction, current: prediction)
seed: random seed. (default: -81890329110200490, current: 122334455)
tol: the convergence tolerance for iterative algorithms (>= 0). (default: 0.0001)
weightCol: weight column name. If this is not set or empty, we treat all instance weights as 1.0. (undefined)
Cluster Centers: 
[4.97482656e-01 5.8109

### Testen des Modells

In [136]:
predictions = cvModel.transform(testData)
predictions.show()

+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+--------------------+----------+
|       Bundesland|BundeslandIndex|       Landkreis|LandkreisIndex|Altersgruppe|AltersgruppeIndex|Geschlecht|GeschlechtIndex|FallStatus| Falldatum|FallStatusIndex|            features|     indexedFeatures|prediction|
+-----------------+---------------+----------------+--------------+------------+-----------------+----------+---------------+----------+----------+---------------+--------------------+--------------------+----------+
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTORBEN|2020-04-14|            1.0|[1.0,1.0,2.0,8426...|[1.0,1.0,2.0,8426...|         3|
|Baden-Württemberg|              8|     LK Biberach|          8426|     A60-A79|              2.0|         M|            1.0| GESTOR

In [137]:
# Fläche unter der Soluette
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " , silhouette)

Silhouette with squared euclidean distance =  0.6737277242091063


In [141]:
predictions.groupBy( "prediction","FallStatus", "Altersgruppe", "Geschlecht").count().orderBy("prediction").show()

+----------+----------+------------+----------+-----+
|prediction|FallStatus|Altersgruppe|Geschlecht|count|
+----------+----------+------------+----------+-----+
|         0|   GENESEN|     A00-A04|         W|   20|
|         0| GESTORBEN|        A80+|         W| 1335|
|         0|   GENESEN|     A15-A34|         M|  256|
|         0|   GENESEN|     A35-A59|         W|  573|
|         0|   GENESEN|        A80+|         M|   67|
|         0|   GENESEN|     A05-A14|         W|   67|
|         0|   GENESEN|        A80+|         W|  166|
|         0|   GENESEN|     A60-A79|         W|  262|
|         0| GESTORBEN|        A80+|         M| 1132|
|         0| GESTORBEN|     A15-A34|         M|    5|
|         0|   GENESEN|     A00-A04|         M|   31|
|         0|   GENESEN|     A15-A34|         W|  310|
|         0|   GENESEN|     A35-A59|         M|  491|
|         0| GESTORBEN|     A60-A79|         M|  669|
|         0|   GENESEN|     A60-A79|         M|  248|
|         0| GESTORBEN|     